# Thermodynamic Relations

In [ ]:
import mmf_setup;mmf_setup.nbinit()
%pylab inline --no-import-all
from nbimports import * 
from mmf_hfb import tf_completion as tf
from mmf_hfb.FuldeFerrelState import FFState
from mmf_hfb import homogeneous
import itertools
import sys
import os
import inspect
from os.path import join
import json
from json import dumps
import re
tf.MAX_DIVISION = 200
clear_output()

\begin{align}
k_a &= k + q + dq, \qquad k_b = k+q - dq ,\qquad
\epsilon_a = \frac{\hbar^2}{2m}k_a^2 - \mu_a, \qquad \epsilon_b = \frac{\hbar^2}{2m}k_b^2 - \mu_b,\\
E&=\sqrt{\epsilon_+^2+\abs{\Delta}^2},\qquad \omega_+= \epsilon_-+E, \qquad \omega_- = \epsilon_- - E\\
\epsilon_+&= \frac{\hbar^2}{4m}(k_a^2+k_b^2) - \mu_+= \frac{\hbar^2}{2m}\left[(k+q)^2 + dq^2\right] - \mu_+\\
\epsilon_-&= \frac{\hbar^2}{4m}(k_a^2-k_b^2) - \mu_-=\frac{\hbar^2}{m}(k +q)dq - \mu_-\\
\end{align}

In [ ]:
%pylab inline --no-import-all
from ipywidgets import interact
def f(E, T):
    T = max(T, 1e-32)
    return 1./(1+np.exp(E/T))

@interact(delta=(0, 5, 0.1),mu=(0, 10, 0.2),
          dmu=(0, 5, 0.1),q = (0,5,0.1),
          dq = (0,5,0.1),T=(0, 0.1, 0.01))
def go(delta=0.1, mu=5.0, dmu=0.0,q=0, dq=0, T=0.02):
    k = np.linspace(0, 5, 100)
    hbar = m = kF = 1.0
    kF, eF = np.sqrt(2*mu), mu
    mu_a, mu_b = mu + dmu, mu - dmu
    e_a, e_b = (hbar*(k+q+dq))**2/2/m - mu_a, (hbar*(k+q-dq))**2/2/m - mu_b
    e_p, e_m = (e_a + e_b)/2, (e_a - e_b)/2
    E = np.sqrt(e_p**2+abs(delta)**2)
    w_p, w_m = e_m + E, e_m - E
    f_p = 1 - e_p/E*(f(w_m, T) - f(w_p, T))
    f_m = f(w_p, T) - f(-w_m, T)
    f_a, f_b = (f_p+f_m)/2, (f_p-f_m)/2
    plt.figure(figsize=(10,8))
    plt.subplot(211);plt.grid()
    plt.plot(k, f_a, label='a')
    plt.plot(k, f_b, label='b');plt.legend()
    plt.ylabel('n')
    plt.subplot(212);plt.grid()
    plt.axhline(0, c='y')
    plt.plot(k, w_p/eF, k, w_m/eF)
    plt.xlabel('$k/k_F$')
    plt.ylabel(r'$\omega_{\pm}/\epsilon_F$')

### Energy Density for 1D free Fermi Gas($\Delta=0$)
$$
\mathcal{E}=\frac{\sqrt{2m}}{\pi\hbar}\frac{E_F^{3/2}}{3}\qquad n=\frac{m}{\pi \hbar^2}E_F
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{\sqrt{2m}}{\pi\hbar}\frac{\mu_a^{3/2}+\mu_b^{3/2}}{3}=\frac{\sqrt{2m}}{\pi\hbar}\frac{(\mu+d\mu)^{3/2}+(\mu-d\mu)^{3/2}}{3}
\end{align}

### Energy Density for 2D free Fermi Gas($\Delta=0$)
For single component
$$
\mathcal{E}=\frac{m}{4\pi\hbar^2}E_F^2,\qquad n=\frac{m}{\pi\hbar^2}E_F, \qquad \mathcal{E}=\frac{1}{2}nE_F
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{m}{4\pi\hbar^2}(\mu_a^2+\mu_b^2)=\frac{m}{4\pi\hbar^2}\left((\mu+d\mu)^2+(\mu-d\mu)^2\right)\\
\end{align}

### Energy Density for 3D free Fermi Gas($\Delta=0$)
For single component
$$
\mathcal{E}=\frac{\hbar^2}{10m\pi^2}k_F^5=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}E_F^{5/2}, \qquad n=\frac{k_F^3}{3\pi^2}
$$
Since there are two components, then the overall energy density
\begin{align}
\mathcal{E}
&=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}(\mu_a^{5/2}+\mu_b^{5/2})=\frac{1}{10\pi^2}\frac{(2m)^{3/2}}{\hbar^3}\left((\mu+d\mu)^{5/2}+(\mu+d\mu)^{5/2}\right)\\
\end{align}

## Test Code
* For Free Fermi Gas, use the ananlytical formuleas

In [ ]:
def get_e_n(mu, dmu, dim=1):
    if dim == 1:
        def f(e_F):
            return np.sqrt(2)/np.pi * e_F**1.5/3.0
        ea, eb = f(mu+dmu),f(mu-dmu)
        energy_density = ea + eb
        na, nb=np.sqrt(2 * (mu + dmu))/np.pi,np.sqrt(2 * (mu - dmu))/np.pi
        total_density = na + nb
    elif dim == 2:
        energy_density = ((mu+dmu)**2 + (mu-dmu)**2)/4.0/np.pi
        na, nb = (mu + dmu)/np.pi/2.0,(mu-dmu)/np.pi/2.0
        total_density = mu/np.pi
    elif dim == 3:
        energy_density = ((mu+dmu)**2.5 + (mu-dmu)**2.5)*2.0**1.5/10.0/np.pi**2
        na, nb = ((2.0 * (mu + dmu))**1.5)/6.0/np.pi**2,((2.0 * (mu - dmu))**1.5)/6.0/np.pi**2
        total_density = ((2.0 * (mu + dmu))**1.5 + (2.0 * (mu - dmu))**1.5)/6.0/np.pi**2
    return energy_density, (na, nb)

* The $\Delta_0$ is set to unitary, which may be not good for large $mu$ and $dmu$

In [ ]:
def test_Thermodynamic(mu, dmu,delta0=1, dim=1, k_c=100, q=0, dq=0, T=0.0, dx=1e-6):
    print(f"mu={mu}\tdmu={dmu}\tkc={k_c}\tq={q}\tdq={dq}\tdim={dim}")    
    ff = FFState(mu=mu, delta=delta0, dim=dim, k_c=k_c, T=T, fix_g=False)

    def get_P(mu, dmu):
        delta = ff.solve(mu=mu, dmu=dmu, q=q, dq=dq, a=0.8*delta0, b=1.2*delta0)
        return ff.get_pressure(mu=mu, dmu=dmu, delta=delta, q=q, dq=dq)

    def get_E_n(mu, dmu):
        E = ff.get_energy_density(mu=mu, dmu=dmu, q=q, dq=dq)
        n = sum(ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq))
        return E, n

    def get_ns(mu, dmu):
        return ff.get_densities(mu=mu, dmu=dmu, q=q, dq=dq)
    energy_density, (na, nb) = get_e_n(mu=mu, dmu=dmu, dim=dim)        
    print(f"{dim}D: E={energy_density}\tn_a={na}\tn_b={nb}\tn_p={na+nb}")    
    E1, n1 = get_E_n(mu=mu+dx, dmu=dmu)
    E0, n0 = get_E_n(mu=mu-dx, dmu=dmu)
    print(f"E1={E1.n}\tE0={E0.n}\tn1={n1.n}\tn0={n0.n}")
    n_p = (get_P(mu+dx, dmu) - get_P(mu-dx, dmu))/2/dx
    n_a, n_b = get_ns(mu, dmu)
    n_a_ = (get_P(mu+dx/2, dmu+dx/2) - get_P(mu-dx/2, dmu - dx/2))/2/dx
    n_b_ = (get_P(mu+dx/2, dmu-dx/2) - get_P(mu-dx/2, dmu + dx/2))/2/dx
    print(f"n_a={n_a.n}\tNumerical  n_a={n_a_.n}")
    print(f"n_b={n_b.n}\tNumerical  n_b={n_b_.n}")
    print(f"n_p={n_a.n+n_b.n}\tNumerical  n_p={n_p.n}")
    print(f"mu={mu}\tNumerical mu={((E1-E0)/(n1-n0)).n}")
    assert np.allclose(n_a.n, n_a_.n, rtol=1e-2)
    assert np.allclose(n_b.n, n_b_.n, rtol=1e-2)
    assert np.allclose(n_p.n, (n_a+n_b).n, rtol=1e-2)
    assert np.allclose(mu,((E1-E0)/(n1-n0)).n, rtol=1e-2)
    
def homogeneous_Density(mu, dmu, delta, dim):
    """return density,used for reference"""
    return homogeneous.Homogeneous(dim = dim).get_ns(mus_eff=(mu+dmu, mu-dmu), delta=delta, N_twist=12)

## 1D Cases 

In [ ]:
homogeneous_Density(mu=5, dmu=0.5, delta=1, dim=1)

* the following test case is interesting, it's failed when dmu=q=dq=0, this error goes away when fix g.
* clearly, we can comfirm that, there is an error when fix $C$.
* this can be used as a trace to fix the error.

In [ ]:
dx = 1e-3
dmu = 3.0
e1, n1 = get_e_n(mu=5 + dx, dmu=dmu)
e2, n2 = get_e_n(mu=5 - dx, dmu=dmu)
e1,e2,  n1, n2, (e1-e2)/(sum(n1)-sum(n2))

In [ ]:
test_Thermodynamic(mu=5, dmu=3.0, k_c=200, q=0.0, dq=0.0, dim=1, delta0=1.0)

## $\Delta=0$ Reasoning

$$
\mathcal{E}=\int_0^{\mu_a} \frac{k^2}{2m}dk + \int_0^{\mu_b} \frac{k^2}{2m}dk
$$

In [ ]:
test_Thermodynamic(mu=5, dmu=0.0, k_c=200, q=.1, dq=0.0, dim=1, delta0=1.0)

# 2D Cases

In [ ]:
homogeneous_Density(mu=5, dmu=0.64, delta=1, dim=2)

### Good accuracy

In [ ]:
test_Thermodynamic(mu=10, dmu=0.64, dim=2, q=0, dq=0.01, delta0=1, k_c=200, dx=1e-6)

# 3D Cases:
* the 3D cases are very sensitive to the cutoff k_c

### homogeneous density

In [ ]:
homogeneous_Density(mu=5, dmu=0.64, delta=1, dim=3)

In [ ]:
test_Thermodynamic(delta0=1,mu=9,dmu=1.2000000000000002, k_c=200 ,q=10.7999999999999998,dq=0,dim=3)

In [ ]:
test_Thermodynamic(mu = 5, dmu = 0.24, dim = 3, k_c = 100, q = 0, dq = 0, dx=1e-3)

# Phase Diagram

In [ ]:
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
files = [f for f in os.listdir(join(currentdir,"../")) if re.match(r'1d_phase_map_data*', f)]
#files = ["1d_phase_map_data_20190314165122_20190314165632.txt"]
rets = []
for file in files:
    file = join(currentdir,"../",file)
    if os.path.exists(file):
        with open(file,'r') as rf:
            rets.extend(json.load(rf))
            formattedData = []
#find the dq with maximum pressure
for items in rets:
    mu = items['mu']
    delta0 = items['delta']
    data = items['data']
    q = items['q']
    na = items['na']
    nb = items['nb']
    g = items['g']
    if len(data) > 0:
        dmu,dq, delta, press = data[0]
        for item in data:
            if dmu == item[0]:
                if item[3] > press:
                    dq, press= item[1],item[3]
            else:
                formattedData.append((mu, dmu, delta0, delta, q, dq, press, g, na, nb))
                dmu, dq, delta, press = item
        formattedData.append((mu, dmu, delta0, delta, q, dq, press, g, na, nb))

In [ ]:
data = formattedData
plt.figure(figsize=(20, 5))
plt.subplot(121)
x,y,c,area =[],[],[],[]
ShowNonFFState=True
for item in data:
    mu, dmu, delta0, delta, q, dq, press, g, na, nb = item
    n = na+nb
    if dmu > 0 and  dq> 0 and delta > 0:
        x.append(-1.0 /g)
        y.append(dmu/delta0)
        c.append('r')
        area.append(16)
    elif ShowNonFFState:
        x.append(-1.0 /g)
        y.append(dmu/delta0)
        c.append('b')
        area.append(1)
plt.scatter(x, y, s=area, c=c)
plt.xlabel(r"$-1/(g\sqrt{n_{d\mu=0}})$", fontsize=20)
plt.ylabel(r"$d\mu/\Delta_0$", fontsize=20)
plt.ylim(0,1)

In [ ]:
qs = [0.5]#np.linspace(0,3,4) # q does not change the pressure, you may use q = 1.5 or other values 
dq0 = 7.856742013183863e-5
mu=0.5
dmu = 0.00039283710065919316
delta = 0.0007071067811865476
for q in qs:
    dqs = np.linspace(2,7,10)*dq0
    ff = FFState(mu=mu, dmu=dmu, dim=2, delta=delta, k_c=100)
    ps = [ff.get_pressure(mu=mu, dmu=dmu,q = q, dq=dq).n for dq in dqs]
    clear_output()
    plt.plot(dqs * 1000, ps)